# Question 1


## Using Zero Shot Learning

In [7]:
import pandas as pd
import numpy as np
from langchain_groq.chat_models import ChatGroq
from MakeDataset import X_train,X_test,y_train,y_test


Groq_Token = ''  # Do not share this key with anyone


groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}


Training data shape:  (126, 500, 3)
Testing data shape:  (54, 500, 3)


In [55]:
x_train = pd.read_csv('X_Train_Tsfel.csv')
x_test = pd.read_csv('X_Test_Tsfel.csv')
activity_label_mapping = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

grouped_data = x_train.sort_values(by = 'class').groupby('class')

csv_tsfel_train_first = grouped_data.get_group(1).to_csv()
csv_tsfel_train_second = grouped_data.get_group(2).to_csv()
csv_tsfel_train_third = grouped_data.get_group(3).to_csv()
csv_tsfel_train_fourth = grouped_data.get_group(4).to_csv()
csv_tsfel_train_fifth = grouped_data.get_group(5).to_csv()
csv_tsfel_train_sixth = grouped_data.get_group(6).to_csv()

csv_tsfel_test = x_test[0:10].to_csv()


In [38]:
total_accuracies = 0
total_iterations = 0
num_test_splits = 5

activity_label_mapping = {"WALKING":1,"WALKING_UPSTAIRS":2,"WALKING_DOWNSTAIRS":3,"SITTING":4,"STANDING":5,"LAYING":6}

for i in range(num_test_splits):
    lot_size = 54//num_test_splits
    csv_tsfel_test = x_test[lot_size*i:lot_size*(i+1)].to_csv()
    
    query = f"""
    * You are an activity classification model. 
    * Your task is to analyze a csv string which will have acceleration values in the x, y and z directions and you have to classify the activity value as "WALKING","WALKING_UPSTAIRS","WALKING_DOWNSTAIRS","SITTING","STANDING","LAYING". 
    * You have a {lot_size} number of rows in the csv string. predict one activity for each of those rows.
    * Provide the activity labels ONLY in a space seperated format as the ouput. Do not provide and other text or code or explanation
    
    Csv_string: {csv_tsfel_test}
    """ 


    # To use Groq LLMs 
    model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
    try:
        llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
        answer = llm.invoke(query)
        # print(f'Correct Answer was {y_test}')
        
        
        string_answer = answer.content.split()
        answer = []
        for ans in string_answer:
            answer.append(activity_label_mapping[ans])

        res = 0
        print(f"Correct answer was: {y_test[lot_size*i:lot_size*(i+1)]}")
        print(f'Model predicted: {answer}')
        for j in range(lot_size):
            if y_test[lot_size*i + j] == answer[j]:
                res += 1
        accuracy = res/lot_size
        
        total_accuracies += accuracy
        total_iterations += 1
    except Exception as e:
        print(f"An error occurred: {e}")
print(f'Total Accuracy turned out to be {100*total_accuracies/total_iterations}%')

Correct answer was: [3 1 2 5 5 1 1 5 3 2]
Model predicted: [1, 1, 1, 2, 4, 1, 1, 2, 5, 1]
Correct answer was: [6 5 6 5 6 1 6 5 2 5]
Model predicted: [1, 1, 3, 2, 3, 2, 3, 2, 2, 1]
Correct answer was: [4 3 2 2 1 4 6 4 1 2]
Model predicted: [1, 1, 1, 4, 5, 6, 3, 3, 1, 1]
Correct answer was: [6 2 4 4 3 6 6 3 1 5]
Model predicted: [1, 1, 1, 1, 1, 3, 3, 1, 1, 1]
Correct answer was: [3 2 1 4 4 4 5 1 3 3]
Model predicted: [1, 1, 1, 4, 5, 6, 1, 1, 1, 1]
Total Accuracy turned out to be 17.999999999999996%


## Using Few Shot Learning

In [39]:
total_accuracies = 0
total_iterations = 0
num_test_splits = 5

for i in range(num_test_splits):
    lot_size = 54//num_test_splits
    csv_tsfel_test = x_test[lot_size*i:lot_size*(i+1)].to_csv()
    
    query = f'''
    You are an activity classification model.
    
    **Instructions:**
    1. **Train** a Decision Tree model using the following training data, which includes 6 classes with PCA1 and PCA2 as feature columns and a class as the target value.
    2. **Predict** the class for each row in the test data, which consists of {lot_size} rows with PCA1 and PCA2 as feature columns.
    3. **Output**: Provide only the {lot_size} predicted activity labels as a single line of space-separated integers. Do not include any additional text, explanations, or code.
    
    **Training Data:** for first class
    {csv_tsfel_train_first}
    **Training Data:** for second class
    {csv_tsfel_train_second}
    **Training Data:** for third class
    {csv_tsfel_train_third}
    **Training Data:** for fourth class
    {csv_tsfel_train_fourth}
    **Training Data:** for fifth class
    {csv_tsfel_train_fifth}
    **Training Data:** for sixth class
    {csv_tsfel_train_sixth}
    
    **Test Data:**
    {csv_tsfel_test}
    '''

    # To use Groq LLMs 
    model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
    try:
        llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=1)
        answer = llm.invoke(query)
        # print(f'Correct Answer was {y_test}')
        
        
        answer = list(map(int, answer.content.split()))
        res = 0
        print(f"Correct answer was: {y_test[lot_size*i:lot_size*(i+1)]}")
        print(f'Model predicted: {answer}')
        for j in range(lot_size):
            if y_test[lot_size*i + j] == answer[j]:
                res += 1
        accuracy = res/lot_size
        
        total_accuracies += accuracy
        total_iterations += 1
    except Exception as e:
        print(f"An error occurred: {e}")
print(f'\nTotal Accuracy turned out to be {100*total_accuracies/total_iterations}%')


Correct answer was: [3 1 2 5 5 1 1 5 3 2]
Model predicted: [1, 5, 1, 4, 2, 1, 1, 5, 1, 2]
Correct answer was: [6 5 6 5 6 1 6 5 2 5]
Model predicted: [4, 5, 3, 6, 1, 2, 3, 6, 4, 5]
Correct answer was: [4 3 2 2 1 4 6 4 1 2]
Model predicted: [1, 3, 1, 5, 5, 4, 2, 4, 1, 5]
Correct answer was: [6 2 4 4 3 6 6 3 1 5]
Model predicted: [3, 1, 6, 1, 3, 6, 4, 6, 2, 1]
Correct answer was: [3 2 1 4 4 4 5 1 3 3]
Model predicted: [1, 2, 3, 4, 5, 1, 4, 2, 2, 1]
T
otal Accuracy turned out to be 28.0%


## Which model performs better and why

Few Shot learning turned out to be a better model in general. 
As opposed to zero shot learning, few shot gives some amount of additional data that the LLM can learn from.
This data makes allows the LLM to train its own decision tree and predict values accordingly.
This gives us better results.

# Question 2


# Question 3

## Limitations of Zero Shot Learning

 ZSL tries to recognize activities it has never seen before by using descriptions or general information about them. 
 
 If these descriptions aren't detailed or accurate enough, 
 it can be hard for the system to match them with the actual movement patterns recorded by the accelrometer.

## Limitations of Few Shot Learning

FSL learns from only a few examples of each activity. 

If those examples don’t cover all the different ways the activity might be performed (like different styles or speeds), the system might not learn enough to identify the activity correctly in new situations.

The LLM might also tend to overfit to the data that it has learnt from due to the limit on the data that can be provided to the LLM

# Quesiton 4

In [52]:
grouped_data = x_train.sort_values(by = 'class').groupby('class')
len_sixth_activity = len(grouped_data.get_group(6))
unknown_activity = grouped_data.get_group(6)[['pca1', 'pca2']].to_csv()

In [54]:
num_test_splits = 5
lot_size = len_sixth_activity

query = f'''
You are an activity classification model.

**Instructions:**
1. **Train** a Decision Tree model using the following training data, which includes 5 classes with PCA1 and PCA2 as feature columns and a class as the target value.
2. **Predict** the class for each row in the test data, which consists of {lot_size} rows with PCA1 and PCA2 as feature columns.
3. **Output**: Provide only the {lot_size} predicted activity labels as a single line of space-separated integers. Do not include any additional text, explanations, or code.

**Training Data:** for first class
{csv_tsfel_train_first}
**Training Data:** for second class
{csv_tsfel_train_second}
**Training Data:** for third class
{csv_tsfel_train_third}
**Training Data:** for fourth class
{csv_tsfel_train_fourth}
**Training Data:** for fifth class
{csv_tsfel_train_fifth}
**Training Data:** for sixth class

**Test Data:**
{unknown_activity}
'''
    
model_name = "llama3-70b" 
try:
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=1)
    answer = llm.invoke(query)
    
    print(answer.content)
    
except Exception as e:
    print(f"An error occurred: {e}")

3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3


As we can see, the LLM wrongly predicts the activity that it has never seen before

It is more or less consistent with the wrong answer. 

This might probably be because the LLM has been restricted to classify data from the other 5 classes

Therefore, the LLM tries to match the unknown different class to the nearest common class

# Question 5

In [57]:
# Creating the Random Data
print(x_train)

         pca1      pca2  class
0    3.321976  0.402108      2
1   -6.726024 -3.189344      6
2   -1.273788  3.804777      5
3    3.529102 -0.800639      2
4    2.759309  0.495436      2
..        ...       ...    ...
121  3.900801 -0.642566      1
122 -4.515600  4.547864      4
123 -7.149338 -3.553792      6
124  2.391431  0.445788      2
125  2.341533  0.635007      2

[126 rows x 3 columns]


In [60]:
import joblib
dt=joblib.load('t2_dt1_g.pkl')

In [61]:
y_hat=dt.predict()

TypeError: BaseDecisionTree.predict() missing 1 required positional argument: 'X'